In [ ]:
!pip install torch -Uq #LLM'lerin eğitimi, ince ayarı ve tahmin üretim işlemlerini yapmamıza olanak sağlar
!pip install transformers -Uq #LLM'ler ile iletişimimizi sağlar
!pip install tiktoken -Uq #Metnin kaç token içerdiğini gösterir
!pip install datasets -Uq #Veriyi parçalama ve filtreleme gibi işlemlerde kullanılır
!pip install matplotlib -Uq #Modelin görsel olarak izlediğimiz kütüphane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 146.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 34.8 MB/s eta 0:00:00


## Önceki Ders Kritik

In [ ]:
import json

In [ ]:
with open("tokenizer_1.json", "r") as f:
  vocab = json.load(f)

In [ ]:
text = "the capital of the united states is not london"

In [ ]:
def tokenize(text):
  parts = text.split()
  ids = []
  for part in parts:
    if part in vocab:
      value = vocab[part]
    else:
      value = vocab["<unk>"]
    ids.append(value)
  return ids

In [ ]:
token_ids = tokenize(text)

In [ ]:
token_ids

In [ ]:
reverse_vocab = {id: part for part, id in vocab.items()}

In [ ]:
def detokenize(ids):
  text = ""
  for id in ids:
    part = reverse_vocab[id]
    text += part + " "
  text = text.strip()
  return text

In [ ]:
detokenize(token_ids)

## Yeni Hafta: Subword bazında ayırma

Bu kod greedy (aç gözlü) yaklaşımı ele alır -> en uzun parçayı seçmeyi garanti eder

In [ ]:
class Tokenizer:
  def __init__(self, vocab_path):
    with open(vocab_path, "r", encoding="utf-8") as f:
      self.vocab = json.load(f)                                         # Kelime - ID eşleştirme
      self.reverse_vocab = {v: k for k, v in self.vocab.items()}        #

  def encode(self, text):
    tokens = []

    for word in text.split():                                           # Her kelimeyi ayır
      i = 0                                                             # Kelimeyi alt parçalara böl
      while i < len(word):
        found_match = False
        for j in range(len(word), i, -1):
          subword = word[i:j]
          if subword in self.vocab:
            tokens.append(self.vocab[subword])
            i = j
            found_match = True
            break
        if not found_match:                                            # Eğer kelime sözlükte yoksa
          tokens.append(self.vocab.get("<unk>", 0))
          i += 1
      tokens.append(self.vocab.get(" ", 1))                            # Kelimelerin sonuna boşluk ekle
    if tokens:
      tokens.pop()                                                     # Son boşluğu kaldır
    return tokens

  def decode(self, ids):
    text = ""
    for id in ids:
      text += self.reverse_vocab.get(id, "<unk>")                      # Neden önceki decode kısmında boşluk eklerken burada eklemiyoruz
    return text

##Haftaya Soru

Sözlükteki anlamları aynı olabilir. Ya peki anlamsal olarak aynılar mı?

In [ ]:
Text_1 = "The cat chased the dog"
Text_2 = "The dog chased the dog"

In [ ]:
Text = "Hiçbir yüz güzel değil senin yüzünden"